# **코드 실행전 할일**
 **1. https://www.python.org/downloads/windows/ 접속하여 파이썬 3.8버전이나 3.10버전 설치**\
 **2. https://code.visualstudio.com/ Visual Studio Code설치(옵션)**\
 **3. 윈도우 실행 눌러서 cmd.exe 입력하여 명령프롬포트 실행**\
 **4. python --version 입력하여 파이썬 버전 확인**\
 **5. pip3 install jupyter 명령실행하여 주피터 노트북 설치**\
 **6. 라이브러리 설치**   
> pip install git+https://github.com/junzis/pyopensky --upgrade  
> pip install pandas  
> pip install tqdm  
> pip install scipy  

 **7. python -m notebook 입력하여 주피터 노트북 설치 또는 Visual Studio Code 실행하여 ipynb 파일열기**   

 
 -----------------------------------------------------------------------------------------------------------




### **● 아래 코드 실행시 오류 발생할 것임(최초실행시에만)**
- 아래코드실행하여 오류 발생하게 하고 윈도우 기준 첨부된 secrete.conf 파일을 C:\Users\사용자명\.config\pyopensky\secrete.conf 로 덮어쓰기

In [ ]:
from pyopensky import OpenskyImpalaWrapper

OpenskyImpalaWrapper()


# **● pyopensky 활용 rollcall_replies자료 12시간 단위로 자료 불러오기**
- start_date, end_date 수정 및 delta 값의 hours 값 아래기준으로 변경 가능 하나 장시간으로 설정시 개별시간이 오래걸리고 컴퓨터가 다운될 수 있음
> days	일  
seconds	초  
microseconds	마이크로 초  
milliseconds	밀리 초 (1밀리 초는 1000마이크로 초)  
minutes	분  
hours	시간  
weeks	주 (7일을 의미함)  

- opensky(OpenskyImpalaWrapper) 실행 옵션은 아래와 같다.
>source (str): Source of messages "adsb" or "rollcall"  
start (str): Start of time period with format YYYY-MM-DD HH:MM:SS  
end (str): End of time period with format YYYY-MM-DD HH:MM:SS  
icao24 (str or list): Filter of one or a list of IACO addresses, default to None  
bound (list): Filter latitude and longitude bound with format of [lat1, lon1, lat2, lon2], default to None  
countfirst (bool): Count and print number of records before actual query, default to True  
limit (int): Return only a number of return records, defualt to None  

- path_to_file 의 경로를 미리 만들어둔 PC내 폴더로 지정


# ● TCAS 발생 시간 위치 필터값으로 State 값 가져오기
- 지정경로로 결과값 저장

In [16]:
from datetime import datetime, timedelta
from pyopensky import OpenskyImpalaWrapper
from tqdm import tqdm

opensky = OpenskyImpalaWrapper()

# 시작 시간과 끝 시간 지정
start_date = datetime(2023, 1, 3)
end_date = datetime(2023, 1, 3, 12)

# 저장 간격 지정(days, hours, weeks, minutes 등 지정 가능)
delta = timedelta(hours=12)


# 시간 간격에 따른 쿼리 데이터 획득 및 진행 상황 바 시현
with tqdm(total=int((end_date - start_date) / delta)) as pbar:
    while start_date < end_date:
        # 설정된 시간 간격에 따른 끝 지점 계산
        interval_end_date = min(start_date + delta, end_date)
    
        # 필터에 따른 쿼리 수행(ICAO24, bound 지정 가능)
        df2 = opensky.query(
            source="adsb",
            start=start_date.strftime("%Y-%m-%d %H:%M:%S"),
            end=interval_end_date.strftime("%Y-%m-%d %H:%M:%S"),
            #icao24="ac32f2",    
            bound=[35.5, 126.1, 37.8, 127.8],
        )
    
        # 날짜 범위에 따른 파일명 생성(path_to_file 에 실제 저장하고자 하는 경로 지정)
        file_name = f"adsb_result_{start_date.strftime('%Y%m%d-%H%M')}_{interval_end_date.strftime('%Y%m%d-%H%M')}.csv"
        path_to_file = "E:/opensky/state_raw/" + file_name
    
        # CSV 포맷으로 데이터 저장
        if df2 is None:
            start_date += delta
        else:
            df2.to_csv(path_to_file, index=False)
            start_date += delta
        import pandas as pd

        # CSV 파일 경로(저장 경로 지정)
        csv_file_path = "E:/opensky/state_raw/" + file_name

        # 저장된 CSV 파일 읽기
        df = pd.read_csv(csv_file_path)

        # Unix 시간 값을 년월일 시간 형식으로 변환하여 새로운 열 추가
        df['converted_time'] = pd.to_datetime(df['time'], unit='s').dt.strftime('%m-%d %H:%M:%S')
        df['baroalt_ft'] = df['baroaltitude']*3.2084
        
        # 'converted_time' 열을 'time' 열 바로 옆에 추가
        df = df[['time', 'converted_time'] + [col for col in df.columns if col != 'time']]

        # 변환된 데이터프레임을 새로운 CSV 파일로 저장
        output_csv_file_path = "E:/opensky/state_raw/" + file_name
        df.to_csv(output_csv_file_path, index=False)
        
        # 진행 상황 바 업데이트
        pbar.update(1)
        

* Server connection successful!


  0%|          | 0/1 [00:00<?, ?it/s]

* Obtaining details of the query...
* OpenSky Impala: 153665 of records found.
* Fetching records...
* Records downloaded.


100%|██████████| 1/1 [01:14<00:00, 74.45s/it]
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR:paramiko.transport:Socket exception: 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다 (10054)
ERROR: